<a href="https://colab.research.google.com/github/Adnan5603/brainybeam-task/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perform advanced data cleaning techniques, such as fuzzy string matching for data deduplication.

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz, process

/usr/local/lib/python3.12/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df = pd.read_csv("/tested.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
string_cols = df.select_dtypes(include='object').columns
print("String columns:", list(string_cols))

String columns: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [6]:
for col in string_cols:
    df[col+"_clean"] = (
        df[col]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r'[^a-z0-9 ]', '', regex=True)
    )


In [7]:
def get_matches(query, choices, scorer=fuzz.token_sort_ratio, limit=5, threshold=85):
    results = process.extract(query, choices, scorer=scorer, limit=limit)
    return [match for match, score in results if score >= threshold]

In [8]:
def deduplicate_column(col, threshold=90):
    unique_vals = df[col].dropna().unique()
    mapping = {}

    for val in unique_vals:
        matches = get_matches(val, unique_vals, threshold=threshold)
        if len(matches) > 1:
            standard = sorted(matches)[0]  # pick the first alphabetically
            for m in matches:
                mapping[m] = standard

    return mapping

In [9]:
for col in string_cols:
    mapping = deduplicate_column(col+"_clean", threshold=90)
    df[col+"_deduped"] = df[col+"_clean"].replace(mapping)

In [10]:
df[['Ticket', 'Ticket_clean', 'Ticket_deduped']].head(30)

,Ticket,Ticket_clean,Ticket_deduped
0,330911,330911,330911
1,363272,363272,363272
2,240276,240276,240276
3,315154,315154,315154
4,3101298,3101298,3101298
5,7538,7538,7538
6,330972,330972,330972
7,248738,248738,248738
8,2657,2657,2657
9,A/4 48871,a4 48871,a4 48871


In [11]:
for col in string_cols:
    print(f"\n===== {col} =====")
    display(df[[col, col+"_clean", col+"_deduped"]].head(20))  # show first 20 rows for each column


===== Name =====


,Name,Name_clean,Name_deduped
0,"Kelly, Mr. James",kelly mr james,kelly mr james
1,"Wilkes, Mrs. James (Ellen Needs)",wilkes mrs james ellen needs,wilkes mrs james ellen needs
2,"Myles, Mr. Thomas Francis",myles mr thomas francis,myles mr thomas francis
3,"Wirz, Mr. Albert",wirz mr albert,wirz mr albert
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",hirvonen mrs alexander helga e lindqvist,hirvonen mrs alexander helga e lindqvist
5,"Svensson, Mr. Johan Cervin",svensson mr johan cervin,svensson mr johan cervin
6,"Connolly, Miss. Kate",connolly miss kate,connolly miss kate
7,"Caldwell, Mr. Albert Francis",caldwell mr albert francis,caldwell mr albert francis
8,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",abrahim mrs joseph sophie halaut easu,abrahim mrs joseph sophie halaut easu
9,"Davies, Mr. John Samuel",davies mr john samuel,davies mr john samuel



===== Sex =====


,Sex,Sex_clean,Sex_deduped
0,male,male,male
1,female,female,female
2,male,male,male
3,male,male,male
4,female,female,female
5,male,male,male
6,female,female,female
7,male,male,male
8,female,female,female
9,male,male,male



===== Ticket =====


,Ticket,Ticket_clean,Ticket_deduped
0,330911,330911,330911
1,363272,363272,363272
2,240276,240276,240276
3,315154,315154,315154
4,3101298,3101298,3101298
5,7538,7538,7538
6,330972,330972,330972
7,248738,248738,248738
8,2657,2657,2657
9,A/4 48871,a4 48871,a4 48871



===== Cabin =====


,Cabin,Cabin_clean,Cabin_deduped
0,NaN,nan,nan
1,NaN,nan,nan
2,NaN,nan,nan
3,NaN,nan,nan
4,NaN,nan,nan
5,NaN,nan,nan
6,NaN,nan,nan
7,NaN,nan,nan
8,NaN,nan,nan
9,NaN,nan,nan



===== Embarked =====


,Embarked,Embarked_clean,Embarked_deduped
0,Q,q,q
1,S,s,s
2,Q,q,q
3,S,s,s
4,S,s,s
5,S,s,s
6,Q,q,q
7,S,s,s
8,C,c,c
9,S,s,s
